In [1]:

from __future__ import division
from __future__ import print_function

seed_value= 0

import os
os.environ['PYTHONHASHSEED']=str(seed_value)
import random
random.seed(seed_value)
import numpy as np
np.random.seed(seed_value)
import tensorflow as tf
tf.set_random_seed(seed_value)
from keras import backend as K
import os
import keras
import pickle
import os.path
import matplotlib.pyplot as plt

import keras
from keras.datasets import mnist
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, AveragePooling2D
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.callbacks import LambdaCallback
from keras.callbacks import TensorBoard
from sklearn.preprocessing import LabelBinarizer
from keras import initializers
import keras.backend as K
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score
import time
from tqdm import tqdm

import six.moves.cPickle as pickle
import gzip
import os
import sys
import timeit

Using TensorFlow backend.


In [2]:
dataset='mnist.pkl.gz'
batch_size = 128
num_classes = 10
epochs = 100
lr=0.001

In [3]:
print('... loading data')

# Load the dataset
with gzip.open(dataset, 'rb') as f:
    try:
        train_set, valid_set, test_set = pickle.load(f, encoding='latin1')
    except:
        train_set, valid_set, test_set = pickle.load(f)

... loading data


In [4]:
(x_train, y_train) = train_set
(x_test, y_test) = test_set
(x_val, y_val) = valid_set
img_rows, img_cols = 28, 28

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    x_val = x_val.reshape(x_val.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    x_val = x_val.reshape(x_val.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_val.shape[0], 'validation samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (50000, 28, 28, 1)
50000 train samples
10000 validation samples
10000 test samples


In [5]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape, x_val.shape, y_val.shape

((50000, 28, 28, 1),
 (50000, 10),
 (10000, 28, 28, 1),
 (10000, 10),
 (10000, 28, 28, 1),
 (10000, 10))

In [6]:
x_train = np.pad(x_train, ((0,0),(2,2),(2,2),(0,0)), 'constant')
x_test = np.pad(x_test, ((0,0),(2,2),(2,2),(0,0)), 'constant')
x_val = np.pad(x_val, ((0,0),(2,2),(2,2),(0,0)), 'constant')
x_train.shape, y_train.shape, x_test.shape, y_test.shape, x_val.shape, y_val.shape

((50000, 32, 32, 1),
 (50000, 10),
 (10000, 32, 32, 1),
 (10000, 10),
 (10000, 32, 32, 1),
 (10000, 10))

In [7]:
input_shape = (32, 32, 1)

In [8]:
def build_model(setseed):
    """
    Builds test Keras model for LeNet MNIST
    :param loss (str): Type of loss - must be one of Keras accepted keras losses
    :return: Keras dense model of predefined structure
    """
    input = Input(shape=input_shape)
    conv1 = Conv2D(6, (3,3), activation='relu', kernel_initializer=initializers.lecun_uniform(seed = setseed))(input)
    avg1 = AveragePooling2D()(conv1)
    conv2 = Conv2D(16, (3,3), activation='relu', kernel_initializer=initializers.lecun_uniform(seed = setseed))(avg1)
    avg2 = AveragePooling2D()(conv2)
    flat= Flatten()(avg2)
    dens1=Dense(units=120, activation='relu')(flat)
    dens2=Dense(units=84, activation='relu')(dens1)
    probs=Dense(num_classes, activation='softmax')(dens2)
    
    model = Model(input=input, output=probs)
    model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics = ['accuracy'])

    return model



In [9]:
all_model = [None,None,None]
losses = [None,None,None]

prediction=[]

all_score =[0,0,0]
gr=[]
wr=[]
xwr=[]

for i in range(3):
    np.random.seed(25+i)
    model = build_model(i+2)
    all_model[i]=model
    
for i in range(3):    
    weights = all_model[i].trainable_weights # weight tensors
    weights = [weight for weight in weights] # filter down weights tensors to only ones which are trainable
    gradients = all_model[i].optimizer.get_gradients(all_model[i].total_loss, weights) # gradient tensors
    gr.append(gradients)
    wr.append(weights)
    xweights = all_model[i].non_trainable_weights # weight tensors
    xweights = [weight for weight in xweights] # filter down weights tensors to only ones which are trainable
    xwr.append(xweights)

    losses[i]=all_model[i].total_loss
    prediction.append(all_model[i].output)

C:\Users\ikarabayir\AppData\Local\Continuum\anaconda2\envs\tf\lib\site-packages\ipykernel_launcher.py:17: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


In [10]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 32, 32, 1)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 30, 30, 6)         60        
_________________________________________________________________
average_pooling2d_5 (Average (None, 15, 15, 6)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 13, 13, 16)        880       
_________________________________________________________________
average_pooling2d_6 (Average (None, 6, 6, 16)          0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 576)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 120)               69240     
__________

In [12]:
input_tensors = [all_model[0].inputs[0], # input data
                 all_model[0].sample_weights[0], # how much to weight each sample by
                 all_model[0].targets[0], # labels
                 K.learning_phase(), # train or test mode
                 all_model[1].inputs[0], # input data
                 all_model[1].sample_weights[0], # how much to weight each sample by
                 all_model[1].targets[0], # labels
                 all_model[2].inputs[0], # input data
                 all_model[2].sample_weights[0], # how much to weight each sample by
                 all_model[2].targets[0], # labels
                ]


minlos = K.argmin(losses)

grr=[]
for x in gr:
    for y in x:
        grr.append(y)

upd_test= K.function(inputs=input_tensors, outputs=[ losses[0], losses[1], losses[2], minlos, prediction[0], prediction[1], prediction[2] ])


grad_best=[]
grad_non0 = []
grad_non1 = []


weig_best=[]
weig_non0 = []
weig_non1 = []

xweig_best=[]
xweig_non0 = []
xweig_non1 = []




for i in range(len(gr[0])):
    gr_ck=tf.concat([gr[0][i],gr[1][i], gr[2][i]],0)
    newshape = (3, ) + (tuple(wr[0][i].shape))

    
    gr_ck2=tf.reshape(gr_ck, newshape)
    
    bb = gr_ck2[minlos]
    grad_best.append(bb)
    
    nbb0 = gr_ck2[0:minlos]                       #[0,enk) U (enk,] aralıklarının birleşimi bize nonbesti verecek
    nbb1 = gr_ck2[minlos+1:]                      #[0,enk) U (enk,] aralıklarının birleşimi bize nonbesti verecek
    nbc = tf.concat([nbb0,nbb1], 0)    
    nbc = tf.reshape(nbc, (-1,))
    newshape2 = (2, ) + (tuple(wr[0][i].shape))
    
    nbc2 = tf.reshape(nbc, newshape2) 
    nb0 = nbc2[0]
    nb1 = nbc2[1]
    grad_non0.append(nb0)
    grad_non1.append(nb1)
    

    wr_ck=tf.concat([wr[0][i],wr[1][i], wr[2][i]],0)
    
    newshape = (3, ) + (tuple(wr[0][i].shape))
    wr_ck2=tf.reshape(wr_ck, newshape) 
    bb2 = wr_ck2[minlos]
    weig_best.append(bb2)
    
    #wb = wr_ck[minlos]
    wnbb0 = wr_ck2[0:minlos]                       #[0,enk) U (enk,] aralıklarının birleşimi bize nonbesti verecek
    wnbb1 = wr_ck2[minlos+1:]                      #[0,enk) U (enk,] aralıklarının birleşimi bize nonbesti verecek
    wnbc = tf.concat([wnbb0,wnbb1],0)    
    wnbc = tf.reshape(wnbc, (-1,))
    newshape2 = (2, ) + (tuple(wr[0][i].shape))
    
    wnbc2 =tf.reshape(wnbc, newshape2)
    wnb0 = wnbc2[0]
    wnb1 = wnbc2[1]
    weig_non0.append(wnb0)
    weig_non1.append(wnb1)
    
    if i<len(xwr[0]):
        print (i)
        xwr_ck=tf.concat([xwr[0][i],xwr[1][i], xwr[2][i]], 0)

        newshape = (3, ) + (tuple(xwr[0][i].shape))
        
        xwr_ck2=tf.reshape(xwr_ck, newshape)  
        xbb2 = xwr_ck2[minlos]
        xweig_best.append(xbb2)

        #wb = wr_ck[minlos]
        xwnbb0 = xwr_ck2[0:minlos]                       #[0,enk) U (enk,] aralıklarının birleşimi bize nonbesti verecek
        xwnbb1 = xwr_ck2[minlos+1:]                      #[0,enk) U (enk,] aralıklarının birleşimi bize nonbesti verecek
        xwnbc = tf.concat([xwnbb0,xwnbb1], 0)    
        
        xwnbc = tf.reshape(xwnbc, (-1,))
        newshape2 = (2, ) + (tuple(xwr[0][i].shape))
         
        xwnbc2 = tf.reshape(xwnbc, newshape2) 
        xwnb0 = xwnbc2[0]
        xwnb1 = xwnbc2[1]
        xweig_non0.append(xwnb0)
        xweig_non1.append(xwnb1)
    else:
        pass

    
los=tf.stack([losses[0], losses[1], losses[2]])

newshape = (3, )
los2=tf.reshape(los, newshape) 
losbest = los2[minlos]

#wb = wr_ck[minlos]
los_0 = los2[0:minlos]                       #[0,enk) U (enk,] aralıklarının birleşimi bize nonbesti verecek
los_1 = los2[minlos+1:]                      #[0,enk) U (enk,] aralıklarının birleşimi bize nonbesti verecek
loswnbc = tf.concat([los_0,los_1],0)    
loswnbc = tf.reshape(loswnbc,(-1,))
newshape2 = (2, )

loswnbc2 = tf.reshape(loswnbc, newshape2)
losss0 = loswnbc2[0]
losss1 = loswnbc2[1]

agradb = [tf.Variable(tf.zeros(t.shape, dtype=tf.float32, name='m_best')) for t in weig_best]
agradn0 = [tf.Variable(tf.zeros(t.shape, dtype=tf.float32, name='m_0')) for t in weig_non0]
agradn1 = [tf.Variable(tf.zeros(t.shape, dtype=tf.float32, name='m_1')) for t in weig_non1]

step_size = 0.01
eps = 0.000001

upd2=[]

for agrad, best, gbest,  param_i, in zip(agradb, weig_best, grad_best, wr[2]):
    accgrad = agrad + gbest * gbest
    dx = - (step_size / tf.sqrt(accgrad + eps)) * gbest
    # accgrad = tf.cast(accgrad, tf.float32)
    # dx = tf.cast(dx, tf.float32)    
    upd2.extend([tf.assign(agrad, accgrad)])
    upd2.extend([tf.assign(param_i, best + dx)])


upd2.extend([tf.assign(param_i, v)
        for param_i, v in zip(xwr[2], xweig_best)]
    )    
    
    
upd_bb2= K.function(inputs=input_tensors, outputs=[ losses[0], losses[1], losses[2], minlos, prediction[0], prediction[1], prediction[2] ], updates=upd2)






lossepoch=[]
lossepoch_test=[]
lossx=[]
acctra=[]
loss_test=[]
acc_test=[]
skip=[]


loss_val=[]
acc_val=[]
lossepoch_val=[]

   
for f in tqdm(range(epochs)):
    program_starts = time.time()
    tr1=[]
    tr2=[]
    res1=[]
    res2=[]
    res3=[]
    res4=[]
    print('Epoch', f)
    print ('train')
    
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:

    batches = 0
    for x_batch, y_batch in ImageDataGenerator().flow(x_train, y_train, batch_size=batch_size):
        K.set_learning_phase(1)
        inputs = [x_batch, # X
                  np.ones(y_batch.shape[0]), # sample weights
                  y_batch, # y
                  1, # learning phase in Train mode
                  x_batch, # X
                  np.ones(y_batch.shape[0]), # sample weights
                  y_batch, # y
                  x_batch, # X
                  np.ones(y_batch.shape[0]), # sample weights
                  y_batch, # y
                 ]
        ll = upd_bb2(inputs)
        yhat=ll[6]
        #print (accuracy_score(np.argmax(y_batch,axis=1), np.argmax(yhat,axis=1)))
        #print (ll[:4])
        lossepoch.append(ll[2])
        tr1.append(ll[2])
        tr2.append(accuracy_score(np.argmax(y_batch,axis=1), np.argmax(yhat,axis=1)))
        skip.append(ll[3])
        batches += 1
        if batches > len(x_train) / batch_size:
            # we need to break the loop by hand because
            # the generator loops indefinitely
            break
    m=(len(x_train) / batch_size)-int((len(x_train) / batch_size))
    tr1[-1]*=m
    tr2[-1]*=m
    lossx.append(np.mean(tr1))
    acctra.append(np.mean(tr2))
    print ('train loss score is :'+str(np.mean(tr1)))
    print ('train acc score is :'+str(np.mean(tr2)))
    now = time.time()
    print("It has been {0} seconds since the loop started".format(now - program_starts))
    print ('test')
    batchesx = 0
    
    
    
    
    print ('validation')
    batchesx = 0
    for x_batch, y_batch in ImageDataGenerator().flow(x_val, y_val, batch_size=batch_size):
        K.set_learning_phase(0)
        inputs = [x_batch, # X
                  np.ones(y_batch.shape[0]), # sample weights
                  y_batch, # y
                  1, # learning phase in VAl mode
                  x_batch, # X
                  np.ones(y_batch.shape[0]), # sample weights
                  y_batch, # y
                  x_batch, # X
                  np.ones(y_batch.shape[0]), # sample weights
                  y_batch, # y
                 ]
        ll = upd_test(inputs)
        yhat=ll[6]
        #print (accuracy_score(np.argmax(y_batch,axis=1), np.argmax(yhat,axis=1)))
        #print (ll[:3])
        lossepoch_val.append(ll[2])
        res3.append(ll[2])
        res4.append(accuracy_score(np.argmax(y_batch,axis=1), np.argmax(yhat,axis=1)))
        batchesx += 1
        if batchesx >= len(x_val) / batch_size:
            break
    m=(len(x_val) / batch_size)-int((len(x_val) / batch_size))
    res3[-1]*=m
    res4[-1]*=m
    loss_val.append(np.mean(res3))
    acc_val.append(np.mean(res4))
    print ('val loss score is :'+str(np.mean(res3)))
    print ('val acc score is :'+str(np.mean(res4)))
    print ('test')
    batchesx = 0
    for x_batch, y_batch in ImageDataGenerator().flow(x_test, y_test, batch_size=batch_size):
        K.set_learning_phase(0)
        inputs = [x_batch, # X
                  np.ones(y_batch.shape[0]), # sample weights
                  y_batch, # y
                  1, # learning phase in TEST mode
                  x_batch, # X
                  np.ones(y_batch.shape[0]), # sample weights
                  y_batch, # y
                  x_batch, # X
                  np.ones(y_batch.shape[0]), # sample weights
                  y_batch, # y
                 ]
        ll = upd_test(inputs)
        yhat=ll[6]
        #print (accuracy_score(np.argmax(y_batch,axis=1), np.argmax(yhat,axis=1)))
        #print (ll[:3])
        lossepoch_test.append(ll[2])
        res1.append(ll[2])
        res2.append(accuracy_score(np.argmax(y_batch,axis=1), np.argmax(yhat,axis=1)))
        batchesx += 1
        if batchesx >= len(x_test) / batch_size:
            break
    m=(len(x_test) / batch_size)-int((len(x_test) / batch_size))
    res1[-1]*=m
    res2[-1]*=m
    loss_test.append(np.mean(res1))
    acc_test.append(np.mean(res2))
    print ('test loss score is :'+str(np.mean(res1)))
    print ('test acc score is :'+str(np.mean(res2)))



  0%|                                                                                          | 0/100 [00:00<?, ?it/s]

Epoch 0
train
Using real-time data augmentation.
train loss score is :0.2894153698797688
train acc score is :0.9085477941176471
It has been 9.507044792175293 seconds since the loop started
test
validation
val loss score is :0.11579444091413406
val acc score is :0.9569818037974683
test
test loss score is :0.11207490822276737
test acc score is :0.9562895569620253


  1%|▊                                                                                 | 1/100 [00:11<18:17, 11.08s/it]

Epoch 1
train
Using real-time data augmentation.
train loss score is :0.10777357495997263
train acc score is :0.9666719948849105
It has been 6.432597637176514 seconds since the loop started
test
validation
val loss score is :0.09201955739880287
val acc score is :0.9631131329113924
test
test loss score is :0.08297060427102673
test acc score is :0.963310917721519


  2%|█▋                                                                                | 2/100 [00:18<16:31, 10.12s/it]

...

 98%|███████████████████████████████████████████████████████████████████████████████▍ | 98/100 [12:13<00:15,  7.79s/it]

Epoch 98
train
Using real-time data augmentation.
train loss score is :0.003540114502643135
train acc score is :0.99838155370844
It has been 6.288406610488892 seconds since the loop started
test
validation
val loss score is :0.053123326187146686
val acc score is :0.9764636075949367
test
test loss score is :0.03944807615080438
test acc score is :0.978935917721519


 99%|████████████████████████████████████████████████████████████████████████████████▏| 99/100 [12:21<00:07,  7.76s/it]

Epoch 99
train
Using real-time data augmentation.
train loss score is :0.0034909197924640196
train acc score is :0.9984215153452686
It has been 6.187417030334473 seconds since the loop started
test
validation
val loss score is :0.053321933307496336
val acc score is :0.9763647151898734
test
test loss score is :0.03825412956873563
test acc score is :0.9793314873417721


100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [12:29<00:00,  7.72s/it]


In [13]:
#\K.clear_session()

In [14]:
print (np.max(acc_test), np.argmax(acc_test))
print (np.max(acc_val), np.argmax(acc_val))
print (np.min(loss_test), np.argmin(loss_test))
print (np.min(loss_val), np.argmin(loss_val))


0.9797270569620253 84
0.977254746835443 58
0.033674035243442445 29
0.0474609411729932 40


In [15]:
np.savetxt("Adagrad_Mnist_lossepoch.csv", lossepoch, delimiter=",", fmt='%s')
np.savetxt("Adagrad_Mnist_lossepoch_test.csv", lossepoch_test, delimiter=",", fmt='%s')
np.savetxt("Adagrad_Mnist_loss_tra.csv", lossx, delimiter=",", fmt='%s')
np.savetxt("Adagrad_Mnist_skip.csv", skip, delimiter=",", fmt='%s')
np.savetxt("Adagrad_Mnist_acc_tra.csv", acctra, delimiter=",", fmt='%s')
np.savetxt("Adagrad_Mnist_loss_test.csv", loss_test, delimiter=",", fmt='%s')
np.savetxt("Adagrad_Mnist_acc_test.csv", acc_test, delimiter=",", fmt='%s')